In [ ]:
%load_ext lab_black

Step: Load data

In [14]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import numpy as np

In [11]:

url = 'https://www.pro-football-reference.com/years/2020/passing.htm'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'passing'})

# Extract the headers
headers = [th.text for th in table.thead.find_all('th') if th.text != 'Rk']

# Extract the rows
rows = table.tbody.find_all('tr')

# Extract the data from each row
data = []
for row in rows:
    if row.get('class') and 'thead' in row['class']:
        continue
    data_row = [td.text for td in row.find_all('td')]
    data.append(data_row)

In [32]:
def scrape_table(url, table_id):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    table = soup.find('table', {'id': table_id})
    header_rows = table.thead.find_all('tr')
    header_row = header_rows[-1]  # Use the last header row to get the correct headers
    headers = [th.text for th in header_row.find_all('th') if th.text != 'Rk']

    rows = table.tbody.find_all('tr')
    data = []
    for row in rows:
        if row.get('class') and 'thead' in row['class']:
            continue
        data_row = [td.text for td in row.find_all('td') if td.get('data-stat') != 'ranker']
        data.append(data_row)

    return pd.DataFrame(data, columns=headers)


year = 2020
base_url = 'https://www.pro-football-reference.com/years/{}/'

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_table(url, table_id):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    table = soup.find('table', {'id': table_id})
    headers = [th.text for th in table.thead.find_all('th') if th.text != 'Rk']

    rows = table.tbody.find_all('tr')
    data = []
    for row in rows:
        if row.get('class') and 'thead' in row['class']:
            continue
        data_row = [td.text for td in row.find_all('td')]
        data.append(data_row)

    df = pd.DataFrame(data, columns=headers)
    df = df.dropna(axis=1, how='all')  # Remove empty columns
    return df







In [33]:
# Scrape passing data
passing_url = base_url.format(year) + 'passing.htm'
passing_df = scrape_table(passing_url, 'passing')
passing_df

Passing Data:


,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,Cmp%,...,Y/G,Rate,QBR,Sk,Yds,Sk%,NY/A,ANY/A,4QC,GWD
0,Deshaun Watson*,HOU,25,QB,16,16,4-12-0,382,544,70.2,...,301.4,112.4,63.7,49,293,8.3,7.64,8.22,,
1,Patrick Mahomes*,KAN,25,QB,15,15,14-1-0,390,588,66.3,...,316.0,108.2,78.1,22,147,3.6,7.53,8.33,3,3
2,Tom Brady,TAM,43,QB,16,16,11-5-0,401,610,65.7,...,289.6,102.2,66.0,21,143,3.3,7.12,7.53,3,3
3,Matt Ryan,ATL,35,QB,16,16,4-12-0,407,626,65.0,...,286.3,93.3,59.8,41,257,6.1,6.48,6.52,,
4,Josh Allen*,BUF,24,QB,16,16,13-3-0,396,572,69.2,...,284.0,107.2,76.6,26,159,4.3,7.33,7.82,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,Brett Kern,TEN,34,P,13,0,,0,1,0.0,...,0.0,39.6,0.0,0,0,0.0,0.00,0.00,,
108,D.J. Moore,CAR,23,WR,15,14,,0,1,0.0,...,0.0,39.6,,0,0,0.0,0.00,0.00,,
109,Zach Pascal,IND,26,WR,16,14,,0,1,0.0,...,0.0,39.6,1.8,0,0,0.0,0.00,0.00,,
110,Sammy Watkins,KAN,27,WR,10,9,,0,1,0.0,...,0.0,0.0,0.0,0,0,0.0,0.00,-45.00,,


In [34]:
# Scrape rushing data
rushing_url = base_url.format(year) + 'rushing.htm'
rushing_df = scrape_table(rushing_url, 'rushing')
rushing_df

,Player,Tm,Age,Pos,G,GS,Att,Yds,TD,1D,Lng,Y/A,Y/G,Fmb
0,Derrick Henry *+,TEN,26,RB,16,16,378,2027,17,98,94,5.4,126.7,3
1,Dalvin Cook*,MIN,25,RB,14,14,312,1557,16,91,70,5.0,111.2,5
2,Josh Jacobs*,LVR,22,RB,15,15,273,1065,12,61,28,3.9,71.0,2
3,David Montgomery,CHI,23,RB,15,14,247,1070,8,59,80,4.3,71.3,1
4,Ezekiel Elliott,DAL,25,RB,15,15,244,979,6,62,31,4.0,65.3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,Jonathan Williams,DET,26,RB,5,0,1,5,0,0,5,5.0,1.0,1
368,Mike Williams,LAC,26,WR,15,11,1,1,0,0,1,1.0,0.1,0
369,Javon Wims,CHI,26,WR,13,1,1,2,0,0,2,2.0,0.2,0
370,Olamide Zaccheaus,ATL,23,WR,11,2,1,0,0,0,0,0.0,0.0,0


In [35]:
# Scrape receiving data
receiving_url = base_url.format(year) + 'receiving.htm'
receiving_df = scrape_table(receiving_url, 'receiving')
receiving_df

,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
0,Stefon Diggs*+,BUF,27,WR,16,15,166,127,76.5%,1535,12.1,8,73,55,9.2,7.9,95.9,0
1,Davante Adams*+,GNB,28,WR,14,14,149,115,77.2%,1374,11.9,18,73,56,9.2,8.2,98.1,1
2,DeAndre Hopkins*,ARI,28,WR,16,16,160,115,71.9%,1407,12.2,6,75,60,8.8,7.2,87.9,3
3,Darren Waller*,LVR,28,TE,16,15,145,107,73.8%,1196,11.2,9,68,38,8.2,6.7,74.8,2
4,Travis Kelce*+,KAN,31,TE,15,15,145,105,72.4%,1416,13.5,11,79,45,9.8,7.0,94.4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,Matthew Stafford,DET,32,QB,16,16,1,0,0.0%,0,,0,0,0,0.0,0.0,0.0,2
521,Taywan Taylor,CLE,25,WR,3,0,1,0,0.0%,0,,0,0,0,0.0,0.0,0.0,
522,Noah Togiai,IND,23,TE,4,0,1,0,0.0%,0,,0,0,0,0.0,0.0,0.0,
523,Eric Tomlinson,2TM,28,TE,7,2,1,0,0.0%,0,,0,0,0,0.0,0.0,0.0,


In [36]:
# Scrape defense data
defense_url = base_url.format(year) + 'defense.htm'
defense_df = scrape_table(defense_url, 'defense')
defense_df

,Player,Tm,Age,Pos,G,GS,Int,Yds,TD,Lng,...,FR,Yds,TD,Sk,Comb,Solo,Ast,TFL,QBHits,Sfty
0,Zach Cunningham,HOU,26,LILB,16,16,0,0,0,0,...,0,0,0,3.0,164,106,58,7,4,
1,Roquan Smith,CHI,23,LILB,16,16,2,16,0,11,...,1,0,0,4.0,139,98,41,18,6,
2,Devin White,TAM,22,LILB,15,15,0,0,0,0,...,1,2,0,9.0,140,97,43,15,16,
3,Neville Hewitt,NYJ,27,RILB,16,16,0,0,0,0,...,1,17,0,2.0,134,91,43,6,4,
4,Jordan Poyer,BUF,29,SS,16,16,2,14,0,14,...,0,0,0,2.0,124,91,33,4,4,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358,Jeff Wilson,SFO,25,RB,12,3,,,0,,...,1,0,0,0.0,,,,,,
1359,Javon Wims,CHI,26,WR,13,1,,,0,,...,,,0,0.0,1,0,1,0,0,
1360,Isaiah Wynn,NWE,25,LT,10,10,,,0,,...,1,0,0,0.0,,,,,,
1361,T.J. Yeldon,BUF,27,RB,3,0,,,0,,...,1,0,0,0.0,,,,,,


Step: Change % to actual number